In [ ]:
# Load in Azure Packages
from azureml.core import Workspace, Dataset, Datastore
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails
from azureml.opendatasets import OjSalesSimulated
from azureml.automl.core.forecasting_parameters import ForecastingParameters

In [ ]:
# Load in non-Azure Packages
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [ ]:
# Connect to your AMLS Workspace
ws = Workspace.from_config()

In [ ]:
# Set your Compute Cluster
compute_name = 'compute-cluster'
compute_target = ComputeTarget(ws, compute_name)

In [ ]:
# Set your Datastore
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [ ]:
# Retrieve the OJ Sales Data Set
oj_sales_files = OjSalesSimulated.get_file_dataset()
oj_sales = oj_sales_files.take(10)

In [ ]:
# Create a folder called OJ_Sales
folder_name = "OJ_Sales"
os.makedirs(folder_name, exist_ok=True)

In [ ]:
# Download the OJ Sales Files to your OJ_Sales folder
oj_sales.download(folder_name, overwrite=True)

In [ ]:
# Create a single Pandas dataframe out of your OJ Files
OJ_file_path = Path('OJ_Sales').rglob('*.csv')
OJ_files = [x for x in OJ_file_path]
df = pd.concat((pd.read_csv(f) for f in OJ_files))

In [ ]:
# View your Pandas dataframe
df.head(10)

In [ ]:
# Register OJ Sales as a dataset
Dataset.Tabular.register_pandas_dataframe(df, datastore,
                            "OJ Sales Sample")

In [ ]:
# Set your experiment that will be used to train your AutoML Model
experiment_name = 'OJ-Sales-Forecasting'
exp = Experiment(workspace=ws, name=experiment_name)

In [ ]:
# Retrieve your OJ Sales Sample dataset
dataset_name = "OJ Sales Sample"
dataset = Dataset.get_by_name(ws, dataset_name, version='latest')

In [ ]:
# Set your target column
target_column = 'Quantity'

In [ ]:
# Set your task to forecasting
task = 'forecasting'

In [ ]:
# Set the metric you will use to score your model
primary_metric = 'normalized_root_mean_squared_error'

In [ ]:
# Set featurization to auto
featurization = 'auto'

In [ ]:
# Set forecasting parameters
params=ForecastingParameters.from_parameters_dict( {'country_or_region_for_holidays':'US',\
                                                    'drop_columns_names':'Revenue',\
                                                    'forecast_horizon': 6,\
                                                    'target_rolling_window_size': 'auto',\
                                                    'target_lags': 'auto',\
                                                    'feature_lags': 'auto',\
                                                    'seasonality': 'auto',\
                                                    'short_series_handling': True,\
                                                    'use_stl': 'season_trend',\
                                                    'time_column_name':'WeekStarting',\
                                                    'time_series_id_column_names':['Store','Brand'],\
                                                    'short_series_handling_configuration': 'auto'},\
                                                    validate_params=True)

In [ ]:
# Set AutoML configuration
config = AutoMLConfig(task=task,
                     primary_metric=primary_metric,
                     featurization=featurization,
                     compute_target=compute_target,
                     training_data=dataset,
                     label_column_name=target_column,
                     experiment_timeout_minutes=15,
                     enable_early_stopping=True,
                     n_cross_validations=3,
                     model_explainability=True,
                     enable_stack_ensemble=False,
                     enable_voting_ensemble=True,
                     forecasting_parameters=params)

In [ ]:
# Train your AutoML model to forecast OJ Sales
AutoML_run = exp.submit(config, show_output = True)
RunDetails(AutoML_run).show()

In [ ]:
# Set forecasting parameters to train ARIMA and Prophet Models
params2=ForecastingParameters.from_parameters_dict({'country_or_region_for_holidays':None,\
                                                    'drop_columns_names':['Revenue','Price','Advert'],\
                                                    'forecast_horizon': 6,\
                                                    'target_rolling_window_size': None,\
                                                    'target_lags': None,\
                                                    'feature_lags': None,\
                                                    'seasonality': 'auto',\
                                                    'short_series_handling': True,\
                                                    'use_stl': 'season_trend',\
                                                    'time_column_name':'WeekStarting',\
                                                    'time_series_id_column_names':['Store','Brand'],
                                                    'short_series_handling_configuration': 'auto'},\
                                                    validate_params=True)

In [ ]:
# Set AutoML configuration passing in new forecasting parameters
config2=AutoMLConfig(task=task,
                     primary_metric=primary_metric,
                     featurization=featurization,
                     compute_target=compute_target,
                     training_data=dataset,
                     label_column_name=target_column,
                     experiment_timeout_minutes=15,
                     enable_early_stopping=True,
                     n_cross_validations=3,
                     model_explainability=True,
                     enable_stack_ensemble=False,
                     enable_voting_ensemble=True,
                     forecasting_parameters=params2)

In [ ]:
# Train your AutoML model to forecast OJ Sales using Prophet and ARIMA
AutoML_Prophet_ARIMA_run = exp.submit(config, show_output = True)
RunDetails(AutoML_Prophet_ARIMA_run).show()

In [ ]:
# Set model description, tags, and give it a name
description = 'Best AutoML Forecasting Run using OJ Sales Sample Data.' 
tags = {'project' : "OJ Sales", "creator" : "your name"} 
model_name = 'OJ-Sales-Sample-Forecasting-AutoML' 

In [ ]:
# Register your model
AutoML_run.register_model(model_name=model_name, description=description, tags=tags)

In [ ]:
# Reregister your model using a different metric
description = 'Best AutoML Forecasting Run using OJ Sales Sample Data.'
tags = {'project' : "OJ Sales", "creator" : "your name", "metric" : "R2 Score"} 
model_name = 'OJ-Sales-Sample-Forecasting-AutoML-R2'
AutoML_run.register_model(model_name=model_name, description=description, tags=tags, metric = 'r2_score')

In [ ]:
# Retrieve and register your second training run
experiment_name = 'OJ-Sales-Forecasting'
exp = Experiment(workspace=ws, name=experiment_name) 
AutoML_run = AutoMLRun(experiment = exp, run_id = 'your_run_id’) 
description = 'AutoML Forecasting Run for OJ Sales Data with Prophet/ARIMA.'
tags = {'project' : "OJ Sales", "creator" : "your name"} 
model_name = 'OJ-Sales-Sample-Forecasting-AutoML-ARIMA'
AutoML_run.register_model(model_name=model_name, description=description, tags=tags)